In [32]:
#Prediction
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
# Load the model

In [33]:
#load Trained model
model = load_model("churn_model.keras")

with open("label_encoder_gender.pkl", "rb") as f:
    label_encoder_gender = pickle.load(f)
with open("geo_encoder.pkl", "rb") as f:
    onehot_encoder_geo = pickle.load(f)

with open("scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

f:\Class 12-13\Churn\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [34]:
#dummy data
input_data = {
    'CreditScore' : 0,
    'Geography': 'France',
    'Gender' : 'Male',
    'Age': 90,
    'Tenure': 1,
    'Balance': 60,
    'NumOfProducts': 1,
    'HasCrCard': 0,
    'IsActiveMember' : 1,
    'EstimatedSalary': 50
}

In [35]:
geo_encoder = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_df = pd.DataFrame(geo_encoder, columns=onehot_encoder_geo.get_feature_names_out(["Geography"]))

f:\Class 12-13\Churn\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [36]:
geo_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [37]:
#Encode Categorical variables
input_df = pd.DataFrame([input_data])
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,0,France,1,90,1,60,1,0,1,50


In [38]:
#Concat with geo_df
input_df = pd.concat([input_df.drop('Geography', axis=1), geo_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,0,1,90,1,60,1,0,1,50,1.0,0.0,0.0


In [39]:
scalered_input = scaler.transform(input_df)
scalered_input

array([[-6.76262379,  0.91324755,  4.85841541, -1.38944225, -1.2175099 ,
        -0.91668767, -1.54035103,  0.97481699, -1.74529639,  1.00150113,
        -0.57946723, -0.57638802]])

Prediction Churn

In [41]:
prediction = model.predict(scalered_input)
prediction_proba = prediction[0][0]
print("Churn Probability:", prediction_proba)

if prediction_proba > 0.5:
    print("Yes customer bank chor dega")
else:
    print('The customer is not likely to churn')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
Churn Probability: 0.06841327
The customer is not likely to churn
